### This notebook demonstrates the use of post-classification filters created by the ESA Team for their 10m land cover map of Europe.

https://www.mdpi.com/2072-4292/12/21/3523/htm


## Step 0: Load libraries and iniatilize Earth Engine

In [1]:
#Load necessary libraries
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')
sys.path.append('/Users/kristine/Library/Python/3.8/lib/python/site-packages')
import os
import ee
import geemap
import numpy as np
import pandas as pd
from IPython.display import HTML, display
from ipyleaflet import Map, basemaps
import random
import json
import time
import ast

# relative import for this folder hierarchy, credit: https://stackoverflow.com/a/35273613
# module_path = os.path.abspath(os.path.join('..'))
# if module_path not in sys.path:
#     sys.path.append(module_path)
from wri_change_detection import preprocessing as npv
from wri_change_detection import gee_classifier as gclass
from wri_change_detection import post_classification_filters as pcf



<font size="4">Iniatilize Earth Engine and Google Cloud authentication</font>

In [2]:
#Initialize earth engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

<font size="4">Define a seed number to ensure reproducibility across random processes. This seed will be used in all subsequent sampling as well. We'll also define seeds for sampling the training, validation, and test sets.</font>

In [3]:
num_seed=30
random.seed(num_seed)


## Step 1: Load Land Cover Classifications and Label Data


<font size="4">

Define land cover classification image collection, with one image for each time period. Each image should have one band representing the classification in that pixel for one time period.</font>

In [4]:
#Load collection
#This collection represents monthly dynamic world classifications of land cover, later we'll squash it to annual
dynamic_world_classifications_monthly = ee.ImageCollection('projects/wings-203121/assets/dynamic-world/v3-5_stack_tests/wri_test_goldsboro')

#Get classes from first image
dw_classes = dynamic_world_classifications_monthly.first().bandNames()
dw_classes_str = dw_classes.getInfo()

#Get dictionary of classes and values
#Define array of land cover classification values
dw_class_values = np.arange(1,10).tolist()
dw_class_values_ee = ee.List(dw_class_values)
#Create dictionary representing land cover classes and land cover class values
dw_classes_dict = ee.Dictionary.fromLists(dw_classes, dw_class_values_ee)

#Make sure the dictionary looks good
print(dw_classes_dict.getInfo())


{'bare_ground': 8, 'built_area': 7, 'crops': 5, 'flooded_vegetation': 4, 'grass': 3, 'scrub': 6, 'snow_and_ice': 9, 'trees': 2, 'water': 1}


<font size="4">
Load label data to later compare land cover classification to label data. Export points of labelled data in order to compare to classifications later.
</font>

In [27]:
#only labels for regions in Modesto CA, Goldsboro NC, the Everglades in FL, and one region in Brazil have been
#uploaded to this collection
labels = ee.ImageCollection('projects/wri-datalab/DynamicWorld_CD/DW_Labels')

labels_filtered = labels.filterBounds(dynamic_world_classifications_monthly.geometry())
print('Number of labels that overlap classifications', labels_filtered.size().getInfo())

#Compress labels by majority vote
labels_filtered = labels_filtered.reduce(ee.Reducer.mode())
#Remove pixels that were classified as no data
labels_filtered = labels_filtered.mask(labels_filtered.neq(0))
#Rename band
labels_filtered = labels_filtered.rename(['labels'])

#Define geometry to sample points from 
labels_geometry = labels.geometry().bounds()

#Sample points from label image at every pixel
labelPoints = labels_filtered.sample(labels_geometry, scale=scale, projection=crs, 
                                     factor=.1, seed=num_seed, dropNulls=True,
                                     tileScale=8,geometries=True)

labelPoints_export_name = 'goldsboro_smaller'
labelPoints_assetID = 'projects/wri-datalab/DynamicWorld_CD/DW_LabelPoints_{}'
labelPoints_description = 'DW_LabelPoints_{}'

export_results_task = ee.batch.Export.table.toAsset(
    collection=labelPoints, 
    description = labelPoints_description.format(labelPoints_export_name), 
    assetId = labelPoints_assetID.format(labelPoints_export_name))
export_results_task.start()



Number of labels that overlap classifications 3


<font size="4">Define color palettes to map land cover</font>

In [21]:
change_detection_palette = ['#ffffff', # no_data=0
                              '#419bdf', # water=1
                              '#397d49', # trees=2
                              '#88b053', # grass=3
                              '#7a87c6', # flooded_vegetation=4
                              '#e49535', # crops=5
                              '#dfc25a', # scrub_shrub=6
                              '#c4291b', # builtup=7
                              '#a59b8f', # bare_ground=8
                              '#a8ebff', # snow_ice=9
                              '#616161', # clouds=10
]
statesViz = {'min': 0, 'max': 10, 'palette': change_detection_palette};

oneChangeDetectionViz = {'min': 0, 'max': 1, 'palette': ['696a76','ff2b2b']}; #gray = 0, red = 1
consistentChangeDetectionViz = {'min': 0, 'max': 1, 'palette': ['0741df','df07b5']}; #blue = 0, pink = 1



<font size="4">Gather projection and geometry information from the land cover classifications</font>

In [22]:
projection = dynamic_world_classifications_monthly.first().projection().getInfo()
crs = projection.get('crs')
crsTransform = projection.get('transform')
scale = dynamic_world_classifications_monthly.first().projection().nominalScale().getInfo()
print('CRS and Transform: ',crs, crsTransform)

geometry = dynamic_world_classifications_monthly.first().geometry().bounds()


CRS and Transform:  EPSG:32617 [10, 0, 759100, 0, -10, 3930480]


<font size="4">Convert the land cover collection to a multiband image, one band for each year</font>

In [23]:
#Define years to get annual classifications for
years = np.arange(2016,2020)

#Squash scenes from monthly to annual
dynamic_world_classifications = npv.squashScenesToAnnualClassification(dynamic_world_classifications_monthly,years,method='median',image_name='dw_{}')
#Get image names 
dw_band_names = dynamic_world_classifications.aggregate_array('system:index').getInfo()
#Convert to a multiband image and rename using dw_band_names
dynamic_world_classifications_image = dynamic_world_classifications.toBands().rename(dw_band_names)


<font size="4">Map land cover classifications and labels</font>

In [24]:
#Map years to check them out!
center = [35.410769, -78.100163]
zoom = 12
Map1 = geemap.Map(center=center, zoom=zoom,basemap=basemaps.Esri.WorldImagery,add_google_map = False)
Map1.addLayer(dynamic_world_classifications_image.select('dw_2016'),statesViz,name='2016 DW LC')
Map1.addLayer(dynamic_world_classifications_image.select('dw_2017'),statesViz,name='2017 DW LC')
Map1.addLayer(dynamic_world_classifications_image.select('dw_2018'),statesViz,name='2018 DW LC')
Map1.addLayer(dynamic_world_classifications_image.select('dw_2019'),statesViz,name='2019 DW LC')
Map1.addLayer(labels_filtered,statesViz,name='Labels')
display(Map1)


Map(center=[35.410769, -78.100163], controls=(WidgetControl(options=['position'], widget=HBox(children=(Toggle…

<font size="4">Compute a confusion matrix and overall accuracy score for 2019 classifications and the labels</font>

In [14]:
labelPointsFC = ee.FeatureCollection(labelPoints_assetID.format(labelPoints_export_name))

dw_2019 = dynamic_world_classifications_image.select('dw_2019').rename('dw_classifications')

labelPointsWithDW = dw_2019.sampleRegions(collection=label_points, scale=sclae, 
                                                        projection=crs, tileScale=4, geometries=True)




# #sample(region, scale, projection, factor, numPixels, seed, dropNulls, tileScale, geometries)
# #scale=scale, projection=crs

# original_errorMatrix = errorMatrix(labels_masked, dynamic_world_classifications_image.select('dw_2019'))
# #Axis 1 (the rows) of the matrix correspond to the actual values, and Axis 0 (the columns) to the predicted values.
# print(original_errorMatrix.getInfo())
# # accuracy()
# # array()
# # consumersAccuracy()
# # kappa()
# # order()
# # producersAccuracy()


KeyboardInterrupt: 